In [ ]:
# You need to do some weird setup stuff:
# 

In [54]:
!ls -latr waifu-diffusion/logs/

total 4
drwxr-xr-x 15 root root 4096 Sep 18 20:34 ..
drwxr-xr-x  6 root root   90 Sep 18 20:36 2022-09-18T20-35-08_aesthetic
drwxr-xr-x  2 root root    6 Sep 18 21:20 .ipynb_checkpoints
drwxr-xr-x  5 root root  122 Sep 18 22:55 .
drwxr-xr-x  6 root root   90 Sep 18 22:56 2022-09-18T22-54-11_aesthetic


In [47]:
!ls -latr waifu-diffusion/logs/2022-09-18T22-54-11_aesthetic/checkpoints

total 56955504
drwxr-xr-x 6 root root          90 Sep 18 20:36  ..
-rw-r--r-- 1 root root 14580607625 Sep 18 21:31 'epoch=000013.ckpt'
-rw-r--r-- 1 root root 14580607625 Sep 18 21:50 'epoch=000015.ckpt'
-rw-r--r-- 1 root root 14580607625 Sep 18 22:29 'epoch=000020.ckpt'
drwxr-xr-x 2 root root         118 Sep 18 22:29  .
-rw-r--r-- 1 root root 14580607625 Sep 18 22:49  last.ckpt


In [55]:
!cp waifu-diffusion/logs/2022-09-18T22-54-11_aesthetic/checkpoints/last.ckpt pokediffusion_epoch_45.ckpt

In [4]:
import os
import torch

def prune_it(p, keep_only_ema=False):
    print(f"prunin' in path: {p}")
    size_initial = os.path.getsize(p)
    nsd = dict()
    sd = torch.load(p, map_location="cpu")
    print(sd.keys())
    for k in sd.keys():
        if k != "optimizer_states":
            nsd[k] = sd[k]
    else:
        print(f"removing optimizer states for path {p}")
    if "global_step" in sd:
        print(f"This is global step {sd['global_step']}.")
    if keep_only_ema:
        sd = nsd["state_dict"].copy()
        # infer ema keys
        ema_keys = {k: "model_ema." + k[6:].replace(".", ".") for k in sd.keys() if k.startswith("model.")}
        new_sd = dict()

        for k in sd:
            if k in ema_keys:
                new_sd[k] = sd[ema_keys[k]].half()
            elif not k.startswith("model_ema.") or k in ["model_ema.num_updates", "model_ema.decay"]:
                new_sd[k] = sd[k].half()

        assert len(new_sd) == len(sd) - len(ema_keys)
        nsd["state_dict"] = new_sd
    else:
        sd = nsd['state_dict'].copy()
        new_sd = dict()
        for k in sd:
            new_sd[k] = sd[k].half()
        nsd['state_dict'] = new_sd

    fn = f"{os.path.splitext(p)[0]}_pruned.ckpt" if not keep_only_ema else f"{os.path.splitext(p)[0]}-ema-pruned.ckpt"
    print(f"saving pruned checkpoint at: {fn}")
    torch.save(nsd, fn)
    newsize = os.path.getsize(fn)
    MSG = f"New ckpt size: {newsize*1e-9:.2f} GB. " + \
          f"Saved {(size_initial - newsize)*1e-9:.2f} GB by removing optimizer states"
    if keep_only_ema:
        MSG += " and non-EMA weights"
    print(MSG)

In [57]:
prune_it('pokediffusion_epoch_45.ckpt')

prunin' in path: pokediffusion_epoch_45.ckpt
dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])
removing optimizer states for path pokediffusion_epoch_45.ckpt
This is global step 479324.
saving pruned checkpoint at: pokediffusion_epoch_45_pruned.ckpt
New ckpt size: 3.85 GB. Saved 10.73 GB by removing optimizer states


In [ ]:
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!sudo apt install unzip -y
!unzip awscliv2.zip
!sudo ./aws/install

In [7]:
!mkdir ~/.aws
# Upload your ~/.aws/credentials now

!aws configure set default.s3.max_concurrent_requests 3
!aws configure set default.s3.multipart_chunksize 128MB

In [58]:
!aws s3 cp --endpoint-url https://e8c8a19b99810a33b983068eabf3df2f.r2.cloudflarestorage.com/sd-finetune pokediffusion_epoch_45_pruned.ckpt s3://pokediffusion_ckpts_v2

upload: ./pokediffusion_epoch_45_pruned.ckpt to s3://pokediffusion_ckpts_v2/pokediffusion_epoch_45_pruned.ckpt


In [59]:
!aws s3 cp --endpoint-url https://e8c8a19b99810a33b983068eabf3df2f.r2.cloudflarestorage.com/sd-finetune pokediffusion_epoch_45.ckpt s3://pokediffusion_ckpts_v2

upload: ./pokediffusion_epoch_45.ckpt to s3://pokediffusion_ckpts_v2/pokediffusion_epoch_45.ckpt


In [22]:
!rm pokediffusion_epoch_1.ckpt